In [1]:
import tensorflow as tf
from tensorflow import keras
import pickle
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np

In [51]:
model = load_model('churn_model.h5')

# Load the scaler
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)
# Load the label encoder
with open('label_encoder_gen.pkl', 'rb') as f:
    label_encoder_gen = pickle.load(f)
# Load the one-hot encoder
with open('onehot_encoder.pkl', 'rb') as f:
    onehot_encoder = pickle.load(f)


In [42]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [43]:
geo_encoded = onehot_encoder.transform([[input_data['Geography']]])
geo_df = pd.DataFrame(geo_encoded, columns=onehot_encoder.get_feature_names_out()) 
geo_df

d:\Churn_Prediction\Project\venv1\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [44]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [52]:
input_df['Gender'] = label_encoder_gen.transform([input_df['Gender']])
input_df

d:\Churn_Prediction\Project\venv1\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [54]:
# concat the one-hot encoded columns with the original dataframe
input_df = pd.concat([input_df.drop('Geography', axis=1),geo_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [55]:
#scale the input data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [56]:
# predict churn
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 286ms/step


array([[0.03948055]], dtype=float32)

In [61]:
prediction_proba = prediction[0][0]
prediction_proba

np.float32(0.03948055)

In [62]:
if prediction_proba > 0.5:
    print(f"The customer is likely to churn with a probability of {prediction_proba:.2f}")
else:
    print(f"The customer is unlikely to churn with a probability of {prediction_proba:.2f}")

The customer is unlikely to churn with a probability of 0.04
